In [1]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

import utils.utils as utils
import utils.dataloader_utils as dataloader_utils
import utils.train_utils as train_utils

import json
from tqdm import tqdm

In [2]:
tf.config.experimental.list_physical_devices('GPU')

2022-10-24 20:30:19.904154: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


[]

In [3]:
tfp.__version__, tf.__version__

('0.16.0', '2.8.0')

# Load Data

In [4]:
BATCH_SIZE = 1024
intercept = False # we recommand to set this to false to ignore the 0th order of polynomial
num_points_in_one_traj = 11

In [5]:
with open("data/ego_trajs_not_moving_indicies.json", "r") as read_file:
    idx_not_moving = set(json.load(read_file))
    
with open("data/ego_trajs_" + str(num_points_in_one_traj) + "_json/ego_trajs_outlier_indicies.json", "r") as read_file:
    idx_outlier = set(json.load(read_file))

idx_invalid_idx = idx_outlier | idx_not_moving
    
list_dataset = dataloader_utils.generate_file_list_dataset('data/ego_trajs_json/', idx_invalid_idx)
start_idx_dataset = dataloader_utils.generate_start_indicies_dataset("data/ego_trajs_" + str(num_points_in_one_traj) + "_json/ego_trajs_start_point_indicies.json", idx_invalid_idx)
combined_dataset = tf.data.Dataset.zip((list_dataset, start_idx_dataset))

dataProcessor = dataloader_utils.DataProcessor(BATCH_SIZE, combined_dataset, num_points_in_one_traj, traj_type = 'ego_traj', intercept = intercept)
dataProcessor.load_process(shuffle = True)

print(dataProcessor.loaded_dataset.__len__())

100%|██████████| 487002/487002 [00:00<00:00, 3023500.25it/s]


tf.Tensor(377, shape=(), dtype=int64)


In [6]:
len(idx_invalid_idx), len(idx_not_moving), len(idx_outlier)

(101105, 101096, 9)

In [7]:
# Check if the output dimensions are correct
# timestamp has dim [batch_size, num_points_in_one_traj]
# trajectories has dim [batch_size, 2*num_points_in_one_traj]
for timestamp_samples, trajectories_samples in dataProcessor.loaded_dataset:
    print(timestamp_samples.shape, trajectories_samples.shape)
    break

(1024, 11) (1024, 22)


# Analyse ego_xy with observation noise

## Training

In [8]:
EPOCHS = 1
lr = 5e-3

In [11]:
losses = []
best_epoch_losses = []
best_epochs = []
bic_scores = []
aic_scores = []
A_list, B_list = [], []
lr_schedules_ser = []
optimizers_ser = []
log_root_dir = 'logs/gradient_tape/ego_xy' + str(num_points_in_one_traj) + ''
t_scale_factor = (num_points_in_one_traj-1) / 10 # The time duration of one trajectory, for scaling time to interval (0,1)
nan_batches = []
degrees = np.linspace(1, 8, 8, dtype=np.int16) # analyse polynomials from degree 1 to 8
#degrees = [2]
for i_d, deg in enumerate(degrees):
    print('Trainig deg ',deg)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)   
    optimizers_ser.append(tf.keras.optimizers.serialize(optimizer))
    
    if intercept:
        A = tf.Variable(np.random.randn(2*(deg+1), 2*(deg+1)) * 1e-1 , dtype=tf.float64, name='alpha') # Model uncertainty
    else:
        A = tf.Variable(np.random.randn(2*(deg), 2*(deg)) * 1e-1 , dtype=tf.float64, name='alpha') # Model uncertainty
        
    B_diag = tf.Variable(np.random.randn(1) * 1e-1, dtype=tf.float64, name='beta_diag') # Log of Observation uncertainty
    B_by_diag =  tf.Variable(np.random.randn(1) * 1e-1, dtype=tf.float64, name='beta_by_diag')

    
    train_log_dir = log_root_dir + '/deg_' + str(deg)
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)  
   
    model_losses, best_epoch_loss, best_epoch, best_alpha, best_beta_diag, best_beta_by_diag = train_utils.train_ego(alpha=A, beta_diag=B_diag, beta_by_diag=B_by_diag, t_scale_factor = t_scale_factor, degree = deg,
                                                                                                                     opti=optimizer, epochs = EPOCHS, data_loader=dataProcessor.loaded_dataset, tf_summary_writer = train_summary_writer, 
                                                                                                                     verbose = True, early_stop=False)
            
    # Add model loss
    losses.append(model_losses)
    best_epoch_losses.append([best_epoch_loss])
    
    # store the best epoch
    best_epochs.append(best_epoch)
    
    # Compute the AIC and BIC score
    aic_score, bic_score = utils.compute_AIC_BIC(nll = best_epoch_loss, deg = deg, num_points = num_points_in_one_traj)

    bic_scores.append(bic_score)
    aic_scores.append(aic_score)
    
    # Compute the model uncertainty, A_unscaled = np.linalg.inv(scale_mat) @ A_scaled
    A_scale_mat = utils.polyBasisScale(t_scale_factor, deg)
    if not intercept:
        A_scale_mat = A_scale_mat[1:, 1:]
    A_est = np.linalg.inv(np.kron(np.eye(2), A_scale_mat)) @ best_alpha.numpy()
    A_est = A_est @ A_est.T
    A_list.append(A_est)
    
    # Compute the observation uncertainty, B_cov = tf.eye(num_points_in_one_traj) * tf.math.softplus(B)
    B_est = {'B_diag': (tf.math.softplus(best_beta_diag)).numpy(), 
             'B_by_diag': (tf.math.softplus(best_beta_diag) * tf.math.tanh(best_beta_by_diag)).numpy()}
    B_list.append(B_est)
    print(deg, model_losses[-1], bic_score, aic_score)

Trainig deg  1


100%|██████████| 1/1 [00:41<00:00, 41.14s/it]


Epoch  0 , Loss:  93.93805752331583
1 93.93805752331583 99.93279570531176 98.93805752331583
Trainig deg  2


100%|██████████| 1/1 [00:41<00:00, 41.16s/it]


Epoch  0 , Loss:  41.659279838531134
2 41.659279838531134 56.046651475321354 53.659279838531134
Trainig deg  3


...

Epoch  0 , Loss:  22.449081776489574
6 22.449081776489574 118.3648926884244 102.44908177648958
Trainig deg  7


100%|██████████| 1/1 [01:22<00:00, 82.13s/it]


Epoch  0 , Loss:  28.663529925286316
7 28.663529925286316 156.95092701999917 135.66352992528633
Trainig deg  8


100%|██████████| 1/1 [00:45<00:00, 45.22s/it]

Epoch  0 , Loss:  16.309780725474692
8 16.309780725474692 181.76455454856227 154.3097807254747


In [56]:
result = utils.calculate_result(degrees, bic_scores, aic_scores, A_list, B_list, best_epoch_losses, best_epochs, lr, None, EPOCHS, BATCH_SIZE)

In [ ]:
fig,ax = utils.plot_losses(losses, degrees = degrees, y_lim=[-400, 100])
if intercept:
    fig.savefig('imgs/ego_' + str(num_points_in_one_traj) + '_intercept.svg')
else:
    fig.savefig('imgs/ego_' + str(num_points_in_one_traj) + '_new.svg')

In [27]:
result['best_aic_B']

{'B_diag': array([0.00026802]), 'B_by_diag': array([-3.16223622e-06])}

In [ ]:
utils.save_result(folder_dir =log_root_dir, file_name='result_summary', result=result)
with open(log_root_dir + '/' + 'optimizers' + '.json', "w") as write_file:
    json.dump(optimizers_ser, write_file, cls=NumpyEncoder)

# Dummy Code

In [33]:
num_points_in_one_traj = 21
deg_to_change = 1
idx_to_change = deg_to_change-1
with open('logs/gradient_tape/ego_xy' + str(num_points_in_one_traj) + '/result_summary.json', "r") as read_file:
    result_old = json.load(read_file)
          
with open('logs/gradient_tape/ego_xy' + str(num_points_in_one_traj) + '_only_' + str(deg_to_change) + 'th/result_summary.json', "r") as read_file:
    result_sub = json.load(read_file)

In [34]:
result_old['losses'][idx_to_change] = result_sub['losses'][0]
result_old['A_list'][idx_to_change] = result_sub['A_list'][0]
result_old['B_list'][idx_to_change] = result_sub['B_list'][0]

result_old['lr'][idx_to_change] = result_sub['lr'][0]
result_old['optimizer'][idx_to_change] = result_sub['optimizer'][0]

result_old['bic_scores'][idx_to_change] = result_sub['bic_scores'][0]
best_bic_deg_idx = np.where(result_old['bic_scores'] == np.amin(result_old['bic_scores']))[0][0]
result_old['best_bic'] = result_old['bic_scores'][best_bic_deg_idx]
result_old['best_bic_A'] = result_old['A_list'][best_bic_deg_idx]
result_old['best_bic_B'] = result_old['B_list'][best_bic_deg_idx]
result_old['best_bic_deg'] = result_old['degree'][best_bic_deg_idx]
result_old['best_bic_deg_idx'] = best_bic_deg_idx


result_old['aic_scores'][idx_to_change] = result_sub['aic_scores'][0]
best_aic_deg_idx = np.where(result_old['aic_scores'] == np.amin(result_old['aic_scores']))[0][0]
result_old['best_aic'] = result_old['aic_scores'][best_aic_deg_idx]
result_old['best_aic_A'] = result_old['A_list'][best_aic_deg_idx]
result_old['best_aic_B'] = result_old['B_list'][best_aic_deg_idx]
result_old['best_aic_deg'] = result_old['degree'][best_aic_deg_idx]
result_old['best_aic_deg_idx'] = best_aic_deg_idx

with open('logs/gradient_tape/ego_xy' + str(num_points_in_one_traj) + '/result_summary_new.json', "w") as write_file:
    json.dump(result_old, write_file, cls=NumpyEncoder)